In [1]:
import pandas as pd
import csv
import itertools
import collections
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import OrderedDict

In [2]:
file1 = "UserMap.csv"
UserMap = pd.read_csv(file1, names = ['User_ID','Anime_ID','Rating','Name','Genre'], skiprows = 1)
UserMap.head()

,User_ID,Anime_ID,Rating,Name,Genre
0,1,20,6,Naruto,[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
1017680,1,25099,7,Ore ga Ojousama Gakkou ni &quot;Shomin Sample&...,[0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0...
279047,1,5781,7,Isekai no Seikishi Monogatari,[0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1 0 0...
909095,1,20787,8,Black Bullet,[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
1016051,1,24873,6,Juuou Mujin no Fafnir,[0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0...


In [3]:
UserGenreDict = {}
UserSeenMovies = {}
UserFavMovies = {}
for i,rowData in UserMap.iterrows():
    x = rowData['Genre']
    genre = []
    for i in x:
        if (i == '[') or (i == ']') or (i == ' ') or (i == '\r') or (i=='\n'):
            continue
        else:
            genre.append(int(i))
    if rowData['User_ID'] not in UserGenreDict:
        UserGenreDict[rowData['User_ID']] = genre
    else:
        for i in genre:
            UserGenreDict[rowData['User_ID']][i] += genre[i]
    
    if rowData['User_ID'] not in UserSeenMovies:
        UserSeenMovies[rowData['User_ID']] = []
        UserFavMovies[rowData['User_ID']] = []
    UserSeenMovies[rowData['User_ID']].append(rowData['Anime_ID'])
    if rowData['Rating'] > 7:
        UserFavMovies[rowData['User_ID']].append(rowData['Anime_ID'])

In [4]:
def recommendMovies(UserID):
    search = {}
    x = UserGenreDict[UserID]
    x = np.array(x).reshape((1,-1))
    for i in UserGenreDict:
        if i == UserID:
            continue
        else:
            y = UserGenreDict[i]
            y = np.array(y).reshape((1,-1))
            cosineMeasure = cosine_similarity(x,y)
            search[i] = cosineMeasure
    search = OrderedDict(sorted(search.items(), key = lambda x: x[1], reverse = True))
    recommendMovies = []
    count = 0
    for ID in search:
        count += 1
        for i in UserFavMovies[ID]:
            if i not in UserSeenMovies[UserID]:
                recommendMovies.append(i)
        
        if count > 3:
            break
    return recommendMovies

In [5]:
moviesToWatch = recommendMovies(1)

In [10]:
moviesRecommended = UserMap.loc[UserMap['Anime_ID'].isin(moviesToWatch) == True, ('Name')].head(25)

In [11]:
print moviesRecommended

1094617                              Hyouka
1481509         Kuroko no Basket 2nd Season
1515701                            Noragami
1449391                             Btooom!
1325032    Fullmetal Alchemist: Brotherhood
1461473        Magi: The Labyrinth of Magic
1530771                           Haikyuu!!
1610624                       One Punch Man
1109031                         Shaman King
1582369                        Death Parade
1574379                  Shokugeki no Souma
1303454                          Soul Eater
1639915            Hai to Gensou no Grimgar
1441179                  Zetsuen no Tempest
1352248                          Fairy Tail
1506009     Sword Art Online: Extra Edition
1489744                         Gin no Saji
1493728          Magi: The Kingdom of Magic
1461474        Magi: The Labyrinth of Magic
1655543           Full Metal Panic? Fumoffu
1735096               Ookami to Koushinryou
1697306                           Samurai 7
1352249                         

In [13]:
moviesSeen = UserMap.loc[UserMap['Anime_ID'].isin(UserSeenMovies[1]) == True,('Name')]
print moviesSeen

0                                                     Naruto
1017680    Ore ga Ojousama Gakkou ni &quot;Shomin Sample&...
279047                         Isekai no Seikishi Monogatari
909095                                          Black Bullet
1016051                                Juuou Mujin no Fafnir
281566                                    Sora no Otoshimono
875431                       Shingeki no Kyojin: Ano Hi Kara
1077040                                    Grisaia no Rakuen
1012569                               Madan no Ou to Vanadis
601508                Boku wa Tomodachi ga Sukunai Episode 0
573363                   Kore wa Zombie Desu ka? of the Dead
438111                                         Hidan no Aria
686579                         Dakara Boku wa, H ga Dekinai.
505928                                        Ao no Exorcist
1093903                                     Masou Gakuen HxH
1080138                                               ReLIFE
842539                  

In [16]:
print UserGenreDict[57]

[39, 6, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
